# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core import Model 
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from pprint import pprint 
import joblib
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#getting data: the dataset was uploaded to and is accesible through my github repo
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
pdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [3]:
#setting up workspace and experiment
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'expp03ht'

experiment=Experiment(ws, experiment_name)

In [4]:
# setting up IaaS where the experiment will run on it
ccname = "ccp03aml"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists or is in use!, choose a different name")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=ccluster,
        training_data=dshomic2018,
        label_column_name='sexo',
        n_cross_validations=5,
        **automl_settings
)

In [6]:
# TODO: Submit your experiment
exprun = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(exprun).show()
exprun.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of 

        59                                                  0:00:19          nan    0.8805
ERROR: {
    "additional_properties": {},
    "error": {
        "additional_properties": {
            "debugInfo": null
        },
        "code": "UserError",
        "severity": null,
        "message": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_format": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_parameters": {},
        "reference_code": null,
        "details_uri": null,
        "target": null,
        "details": [],
        "inner_error": {
            "additional_properties": {},
            "code": "ResourceExhausted",
            "inner_error": {
                "additional_properties": {},
                "code": "Timeout",
                "inner_error": {
                    "additional_properties": {},
                    "code": "ExperimentTimeoutForIterations",
        

{'runId': 'AutoML_4d4ae00d-b3c4-4fab-9d9b-ad149eaaaa3b',
 'target': 'ccp03aml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-29T10:33:12.783183Z',
 'endTimeUtc': '2021-01-29T11:06:47.974305Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ccp03aml',
  'AMLSettingsJsonString': '{"path":null,"name":"expp03ht","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-136496","workspace_name":"quick-starts-ws-136496","region":"southcentralus","compute_target":"ccp03aml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
#import azureml.train.automl
# obtaining best run and fitted model
brmodel, fmodel = exprun.get_output()

# Print the best run
print(brmodel)

# Get all metrics of the best run model
brmetrics = brmodel.get_metrics()

# Print all metrics of the best run model
for metric_name in brmetrics:
    metric = brmetrics[metric_name]
    print(metric_name, metric)
    

Run(Experiment: expp03ht,
Id: AutoML_4d4ae00d-b3c4-4fab-9d9b-ad149eaaaa3b_62,
Type: azureml.scriptrun,
Status: Completed)
f1_score_macro 0.5020454945811144
f1_score_micro 0.8822466144613257
recall_score_weighted 0.8822466144613257
accuracy 0.8822466144613257
AUC_weighted 0.6183238946164711
weighted_accuracy 0.9809836270291413
precision_score_micro 0.8822466144613257
log_loss 0.36128769225985524
AUC_micro 0.9040676022588008
precision_score_macro 0.8712795045625406
f1_score_weighted 0.8316884986041069
AUC_macro 0.6183238946164712
average_precision_score_weighted 0.8288356587155766
average_precision_score_micro 0.8863526702255718
balanced_accuracy 0.517107879475379
precision_score_weighted 0.879767034650054
matthews_correlation 0.15225145018505634
average_precision_score_macro 0.5675343988516062
recall_score_macro 0.517107879475379
recall_score_micro 0.8822466144613257
norm_macro_recall 0.03421575895075786
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_4d4ae00d-b3c4-4fab-9d9b

In [9]:
# Display parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fmodel)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['8', '39', '18', '1', '54', '13', '0'],
 'weights': [0.1111111111111111,
             0.2222222222222222,
             0.1111111111111111,
             0.2222222222222222,
             0.1111111111111111,
             0.1111111111111111,
             0.1111111111111111]}

8 - sparsenormalizer
{'copy': True, 'norm': 'l2'}

8 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.5,
 'eta': 0.1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 6,
 'max_leaves': 15,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators

In [10]:
import joblib
joblib.dump(value=fmodel, filename='model.pkl')

['model.pkl']

In [11]:
#TODO: Save the best model
autobmlp3ht = brmodel.register_model(model_path='outputs/model.pkl', model_name='p3automl_ht',
                        tags={'Method of execution':'Auto ML'},
                        properties={'Accuracy': brmetrics['accuracy']})

print(autobmlp3ht)

Model(workspace=Workspace.create(name='quick-starts-ws-136496', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-136496'), name=p3automl_ht, id=p3automl_ht:1, version=1, tags={'Method of execution': 'Auto ML'}, properties={'Accuracy': '0.8822466144613257'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
# Download scoring file 
brmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

# Download environment file
brmodel.download_file('outputs/conda_env_v_1_0_0.yml', 'enviro.yml')

In [13]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

infcfg = InferenceConfig(entry_script='scoring.py',
                                    environment=Environment.from_conda_specification(name='myenv',file_path='enviro.yml'))

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wsp03ht01", [autobmlp3ht], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://e22282ab-b5fd-4865-999c-6f8032231e46.southcentralus.azurecontainer.io/score
http://e22282ab-b5fd-4865-999c-6f8032231e46.southcentralus.azurecontainer.io/swagger.json


In [14]:
import json

#Importing the data set for testing 
tsthomic2018 = pdhomic2018.sample(10) 
lblhomic2018 = tsthomic2018.pop('sexo')

tstdatahomic2018 = json.dumps({'data': tsthomic2018.to_dict(orient='records')})

print(tstdatahomic2018)

{"data": [{"id": 1365, "regfalle": 21348, "edad": 24, "deptoocuhe": 10, "tipoarma": 1, "pracaut": 1}, {"id": 457, "regfalle": 12494, "edad": 30, "deptoocuhe": 5, "tipoarma": 1, "pracaut": 1}, {"id": 2868, "regfalle": 23583, "edad": 19, "deptoocuhe": 1, "tipoarma": 1, "pracaut": 1}, {"id": 2219, "regfalle": 23632, "edad": 35, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 0}, {"id": 144, "regfalle": 21066, "edad": 32, "deptoocuhe": 8, "tipoarma": 1, "pracaut": 0}, {"id": 2032, "regfalle": 23671, "edad": 25, "deptoocuhe": 6, "tipoarma": 2, "pracaut": 0}, {"id": 2996, "regfalle": 24526, "edad": 54, "deptoocuhe": 7, "tipoarma": 1, "pracaut": 1}, {"id": 1847, "regfalle": 23264, "edad": 27, "deptoocuhe": 1, "tipoarma": 1, "pracaut": 1}, {"id": 275, "regfalle": 21584, "edad": 22, "deptoocuhe": 2, "tipoarma": 10, "pracaut": 1}, {"id": 1074, "regfalle": 21272, "edad": 18, "deptoocuhe": 7, "tipoarma": 1, "pracaut": 1}]}


TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [16]:
# Print results 
print(response.text)

"{\"result\": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}"


In [17]:
# Print original labels
print(lblhomic2018)

1364    0
456     0
2867    0
2218    0
143     0
2031    0
2995    0
1846    0
274     0
1073    0
Name: sexo, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [18]:
print(webservice.get_logs())

2021-01-29T11:27:18,749393969+00:00 - iot-server/run 
2021-01-29T11:27:18,748775012+00:00 - gunicorn/run 
2021-01-29T11:27:18,751836095+00:00 - rsyslog/run 
2021-01-29T11:27:18,761690507+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4f26b5b9300bb3a3b89415c34dafac63/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [21]:
#best practice: deleting webservice that it won't be used anymore
webservice.delete()

In [22]:
#best practice: deleting CCluster that it won't be used anymore
ccluster.delete()

Current provisioning state of AmlCompute is "Deleting"

